# sat-search

This notebook is a tutorial on how to use sat-search to search STAC APIs, save the results, and download assets.

Sat-search is built using [sat-stac](https://github.com/sat-utils/sat-stac) which provides the core Python classes used to represent STAC catalogs: `Collection`, `Item`, and `Items`. It is recommended to review the [tutorial on STAC Classes](https://github.com/sat-utils/sat-stac/blob/master/tutorial-2.ipynb) for more information on how to use these objects returned from searching.

Only the `search` module is in sat-search is used as a library, and it contains a single class, `Search`. The `parser` module is used for creating a CLI parser, and `main` contains the main function used in the CLI.

**API endpoint**: Sat-search required an endpoint to be passed in or defined by the STAC_API_URL environment variable. This tutorial uses https://earth-search.aws.element84.com/v0 but any STAC endpoint can be used.

## Initializing a Search object

The first step in performing a search is to create a Search object with all the desired query parameters. Query parameters need to follow the querying as provided in the [STAC specification](https://github.com/radiantearth/stac-spec), although an abbreviated form is also supported (see below).

Another place to look at the STAC query format is in the [sat-api docs](http://sat-utils.github.io/sat-api/), specifically see the section on [full-features querying](http://sat-utils.github.io/sat-api/#search-stac-items-by-full-featured-filtering-) which is what sat-search uses to POST queries to an API. Any field that can be provided in the [searchBody](http://sat-utils.github.io/sat-api/#tocssearchbody) can be provided as a keyword parameter when creating the search. These fields include:

- bbox: bounding box of the form [minlon, minlat, maxlon, maxlat]
- intersects: A GeoJSON geometry
- time: A single date-time, a period string, or a range (seperated by /)
- sort: A dictionary of fields to sort along with ascending/descending
- query: Dictionary of properties to query on, supports eq, lt, gt, lte, gte

Examples of queries are in the sat-api docs, but an example JSON query that would be POSTed might be:

```
{
  "bbox": [
    -110,
    39.5,
    -105,
    40.5
  ],
  "time": "2018-02-12T00:00:00Z/2018-03-18T12:31:12Z",
  "query": {
    "eo:cloud_cover": {
      "lt": 10
    }
  },
  "sort": [
    {
      "field": "eo:cloud_cover",
      "direction": "desc"
    }
  ]
}
```

### Simple queries

In sat-search, each of the fields in the query is simply provided as a keyword argument

In [ ]:
import os
os.environ["STAC_API_URL"] = "https://earth-search.aws.element84.com/v0"
url = 'https://earth-search.aws.element84.com/v0'

In [ ]:
os.getenv('STAC_API_URL')

In [ ]:
from satsearch import Search

search = Search(url=url, bbox=[-110, 39.5, -105, 40.5])
print('bbox search: %s items' % search.found())

search = Search(url=url, datetime='2018-02-12T00:00:00Z/2018-03-18T12:31:12Z')
print('time search: %s items' % search.found())

search = Search(url=url, query={'eo:cloud_cover': {'lt': 10}})
print('cloud_cover search: %s items' % search.found())

### Complex query

Now we combine all these filters and add in a sort filter to order the results (which will be shown further below).

In [ ]:
search = Search(url=url, bbox=[-110, 39.5, -105, 40.5],
               datetime='2018-02-12T00:00:00Z/2018-03-18T12:31:12Z',
               query={'eo:cloud_cover': {'lt': 10}},
               collections=['sentinel-s2-l2a'])
print('%s items' % search.found())

### Intersects query

The intersects query works the same way, except a geometry is provided.

In [ ]:
geom = {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -66.3958740234375,
          43.305193797650546
        ],
        [
          -64.390869140625,
          43.305193797650546
        ],
        [
          -64.390869140625,
          44.22945656830167
        ],
        [
          -66.3958740234375,
          44.22945656830167
        ],
        [
          -66.3958740234375,
          43.305193797650546
        ]
      ]
    ]
}

search = Search(url=url, intersects=geom)
print('intersects search: %s items' % search.found())

## Alternate search syntax

This all works fine, except the syntax for creating queries is a bit verbose, so sat-search allows an alternate syntax using simple strings of the property and equality symbols. 

A typical query is shown below for eo:cloud_cover, along with the alternate versions that use the alternate syntax.

In [ ]:
query = {
  "eo:cloud_cover": {
    "lt": 10
  }
}

search = Search(url=url, query=query)
print('%s items found' % search.found())

search = Search.search(url=url, query=["eo:cloud_cover<10"])
print('%s items found' % search.found())

## Fetching results

The examples above use the Search::found() function, but this only returns the total number of hits by performing a fast query with limit=0 (returns no items). To fetch the actual Items use the `Search::items()` function. This returns a sat-stac `Items` object.

In [ ]:
search = Search(url=url, bbox=[-110, 39.5, -105, 40.5],
               datetime='2018-02-01/2018-02-04',
               property=["eo:cloud_cover<5"])
print('%s items' % search.found())

items = search.items()
print('%s items' % len(items))
print('%s collections' % len(items._collections))
print(items._collections)

for item in items:
    print(item)

### Limit

The `search.items()` function does take 1 argument, `limit`. This is the total number of items that will be returned. Behind the scenes sat-search may make multiple queries to the API, up until either the limit, or the total number of hits, whichever is greater.

In [ ]:
items = search.items(limit=2)
print(items.summary())

## Returned `Items`

The returned `Items` object has several useful functions and is covered in detail in the [sat-stac STAC classes tutorial](https://github.com/sat-utils/sat-stac/blob/master/tutorial-2.ipynb). The `Items` object contains all the returned Items (`Items._items`), along with any Collection references by those Items (`Items._collections`), and the search parameters used (`Items._search`) Below are some examples.

In [ ]:
print(items.summary())

In [ ]:
from satstac import ItemCollection

search = Search.search(url=url, bbox=[-110, 39.5, -105, 40.5],
               datetime='2018-02-01/2018-02-10',
               query=["eo:cloud_cover<25"],
               collections=['sentinel-s2-l2a'])
items = search.items()
print(items.summary())

items.save('test.json')
items2 = ItemCollection.open('test.json')

print(items2.summary(['date', 'id', 'eo:cloud_cover']))

In [ ]:
# download a specific asset from all items and put in a directory by date in 'downloads'
filenames = items.download('metadata', filename_template='downloads/${date}/${id}')
print(filenames)